In [1]:
from qiskit.opflow import * 

# Hands on: Opflow 

<img src="OpflowWorkflow.png" width="700"></img>

## 1. Operators and states

The two fundamental objects in Opflow are states, which are of type `StateFn`, and operators, which are of type `OperatorBase`.

States...
* can be based on circuits, vectors or dictionaries (or Operators, see later)
* can be summed
* need not be normalized

In [33]:
from qiskit.circuit import QuantumCircuit
circuit = QuantumCircuit(1)
circuit.x(0)

state = StateFn(circuit)  # this will be of type CircuitStateFn 
print('Circuit:', state)

Circuit: CircuitStateFn(
     ┌───┐
q_0: ┤ X ├
     └───┘
)


In [41]:
vector = [0, 1j]  # amplitude for state |1> is 1
state = StateFn(vector)  # type: VectorStateFn
print('Vector:', state)

Vector: VectorStateFn(Statevector([0.+0.j, 0.+1.j],
            dims=(2,)))


In [40]:
dictionary = {'1': 1}  # probability (or amplitude of state '1' is 1)
state = StateFn(dictionary)  # type: DictStateFn
print('Dict:', state)

Dict: DictStateFn({'1': 1})


In [42]:
state = One  # some frequently used states are predefined in opflow: One Zero Plus Minus
print('Singleton:', state)

Singleton: DictStateFn({'1': 1})


In [43]:
state = Zero + One
print(state)  # caution: state is not normalized

DictStateFn({'0': 1.0, '1': 1.0})


Operators...
* can be summed, tensored, composed and inverted
* don't have to be unitary
* can be based on circuits, matrices or Pauli strings

In [44]:
operator = X  # the Pauli operators and Hadamard are predefined: I, X, Y, Z, H
print('Predefined:')
print(operator.__repr__())
print(operator)

Predefined:
PauliOp(Pauli('X'), coeff=1.0)
X


In [8]:
circuit = QuantumCircuit(2)
circuit.ry(0.2, 0)
circuit.cx(0, 1)
operator = CircuitOp(circuit)
print('Circuit:')
print(operator.__repr__())
print(operator)

Circuit:
CircuitOp(<qiskit.circuit.quantumcircuit.QuantumCircuit object at 0x7fbf393d4c10>, coeff=1.0)
     ┌─────────┐     
q_0: ┤ RY(0.2) ├──■──
     └─────────┘┌─┴─┐
q_1: ───────────┤ X ├
                └───┘


In [9]:
matrix = [[1, 0], [0, 0]]
operator = MatrixOp(matrix)
print('Matrix:')
print(operator.__repr__())
print(operator)

Matrix:
MatrixOp(Operator([[1.+0.j, 0.+0.j],
          [0.+0.j, 0.+0.j]],
         input_dims=(2,), output_dims=(2,)), coeff=1.0)
Operator([[1.+0.j, 0.+0.j],
          [0.+0.j, 0.+0.j]],
         input_dims=(2,), output_dims=(2,))


In [45]:
operator = X.tensor(Y).tensor(Z)  # = X ^ Y ^ Z
print('Tensored:', operator)

operator = X + Y + Z
print('Summed:', operator)

operator = Z.compose(Y)  # = Z @ Y = -iX
print('Composed:', operator)

operator = (X ^ I ^ I) + (Y ^ I ^ I) + ((Z @ Z) ^ I ^ I)
print('All together:', operator)

Tensored: XYZ
Summed: 1.0 * X
+ 1.0 * Y
+ 1.0 * Z
Composed: -iX
All together: 1.0 * XII
+ 1.0 * YII
+ 1.0 * III


We can also specify compositions, sums, or tensors with operators of different types.

In [11]:
circuit = QuantumCircuit(1)
circuit.rx(0.2, 0)
circuit_op = CircuitOp(circuit)

pauli_op = Y

In [12]:
summed = pauli_op + circuit_op + pauli_op
print(summed)

SummedOp([
  Y,
       ┌─────────┐
  q_0: ┤ RX(0.2) ├
       └─────────┘,
  Y
])


In [13]:
composed = summed @ circuit
print(composed)

ComposedOp([
  SummedOp([
    Y,
         ┌─────────┐
    q_0: ┤ RX(0.2) ├
         └─────────┘,
    Y
  ]),
       ┌─────────┐
  q_0: ┤ RX(0.2) ├
       └─────────┘
])


In [14]:
tensored = X ^ circuit
print(tensored)

TensoredOp([
  X,
       ┌─────────┐
  q_0: ┤ RX(0.2) ├
       └─────────┘
])


In [15]:
print(tensored @ (composed ^ H))

ComposedOp([
  TensoredOp([
    X,
         ┌─────────┐
    q_0: ┤ RX(0.2) ├
         └─────────┘
  ]),
  TensoredOp([
    ComposedOp([
      SummedOp([
        Y,
             ┌─────────┐
        q_0: ┤ RX(0.2) ├
             └─────────┘,
        Y
      ]),
           ┌─────────┐
      q_0: ┤ RX(0.2) ├
           └─────────┘
    ]),
         ┌───┐
    q_0: ┤ H ├
         └───┘
  ])
])


Conversion between the different primitives are also supported, if it is possible.

In [51]:
pauli = X @ X @ X
# print(type(pauli))
print(pauli.to_circuit_op())
print(pauli.to_matrix_op())

     ┌───┐
q_0: ┤ X ├
     └───┘
Operator([[0.+0.j, 1.+0.j],
          [1.+0.j, 0.+0.j]],
         input_dims=(2,), output_dims=(2,))


In [17]:
print(pauli.to_matrix_op())

Operator([[-0.70710678+0.j,  0.70710678+0.j],
          [ 0.70710678+0.j,  0.70710678+0.j]],
         input_dims=(2,), output_dims=(2,))


In [52]:
paulisum = X + Y

In [53]:
print(paulisum.to_matrix_op())

Operator([[0.+0.j, 1.-1.j],
          [1.+1.j, 0.+0.j]],
         input_dims=(2,), output_dims=(2,))


In [54]:
# error: not unitary!
print(paulisum.to_circuit_op())

ExtensionError: 'Input matrix is not unitary.'

### Measurements

On paper, we usually write an expectation value as

$$
\langle\psi|\hat O|\psi\rangle.
$$

We can write the same in Opflow

In [55]:
state = One
operator = H

expectation = One.adjoint().compose(H).compose(One)
print(expectation)
print(expectation.eval())

ComposedOp([
  DictMeasurement({'1': 1}),
       ┌───┐
  q_0: ┤ H ├
       └───┘,
  DictStateFn({'1': 1})
])
(-0.7071067811865475+0j)


With Opflow's syntactic sugar we can compress the above, since `~` can be used for adjoint and `@` for composition

In [56]:
expecation = ~One @ H @ One  # this looks a lot like what we have on paper!
print(expectation.eval())

(-0.7071067811865475+0j)


If we measure an expectation value on quantum hardware, however, the mathematical expression does not reflect what really happens. We don't apply the state, then the operator, and then the adjoint of the state. Rather, we prepare the state and then apply a basis transformation such that the operator becomes diagonal in the computational basis before we measure.

So the operations are

$$
\mathrm{Measure}~ \hat T_O |\psi\rangle
$$
where $\hat T_O$ is a basis transformation from the basis of $\hat O$ to the computational basis.

Opflow allows to write expectation values in this fashion as well. 

In [23]:
measurement = StateFn(H, is_measurement=True)
expectation = measurement @ One
print(expectation.eval())

(-0.7071067811865475+0j)


or more concisely

In [24]:
expectation = ~StateFn(H) @ One
print(expectation.eval())

(-0.7071067811865475+0j)


**Note** This is a very important representation and is used for all more complex computations in the Opflow framework.

## 2. Conversions 

### Expectation values

Opflow allows several methods to evaluate expectation values. Above, we used plain numpy matrix multiplication by simply calling `.eval` on the operator expression. This would not work on real hardware, since there we need to evaluate the expectation via basis transformation.

But how do we convert any expectation value to this representation?

The proper way of evaluating expectation values is to go via an `ExpectationBase` converter. This takes an expectation operator as input and returns a new Opflow object ready for evaluation. To prepare the operator for real hardware, we would e.g. use the `PauliExpectation`.

In [25]:
from qiskit.opflow import PauliExpectation

operator = (X ^ X ^ I) + (Z ^ Z ^ I)
state = Plus ^ 3

expectation = ~StateFn(operator) @ state
print(expectation)
print('Reference value:', expectation.eval())

ComposedOp([
  OperatorMeasurement(1.0 * XXI
  + 1.0 * ZZI),
  CircuitStateFn(
       ┌───┐
  q_0: ┤ H ├
       ├───┤
  q_1: ┤ H ├
       ├───┤
  q_2: ┤ H ├
       └───┘
  )
])
Reference value: (1+3.06e-16j)


Let's convert this using the `PauliExpectation`. 

In [26]:
pauli_expectation = PauliExpectation()
converted = pauli_expectation.convert(expectation)
print(converted)
print('Evaluated:', converted.eval())

SummedOp([
  ComposedOp([
    OperatorMeasurement(AbelianSummedOp([
      ZZI
    ])),
    CircuitStateFn(
         ┌───┐     
    q_0: ┤ H ├─────
         ├───┤┌───┐
    q_1: ┤ H ├┤ H ├
         ├───┤├───┤
    q_2: ┤ H ├┤ H ├
         └───┘└───┘
    )
  ]),
  ComposedOp([
    OperatorMeasurement(AbelianSummedOp([
      ZZI
    ])),
    CircuitStateFn(
         ┌───┐
    q_0: ┤ H ├
         ├───┤
    q_1: ┤ H ├
         ├───┤
    q_2: ┤ H ├
         └───┘
    )
  ])
])
Evaluated: (1+3.06e-16j)


Alternatively, we can use
* the `AerPauliExpectation`, which uses Aer's snapshot expectation value and is a very fast, statevector-based operation
* or the `MatrixExpecation`, which is simply a matrix-multiplication evaluation (and not very fast)

### Evolutions

See [Donny's notebook](https://github.com/dongreenberg/aqua_talks/blob/master/Understanding%20Aqua%27s%20Operator%20Flow.ipynb) for more info.

In [59]:
from qiskit.opflow import Suzuki

operator = (X ^ X ^ I) + (Z ^ Z ^ I)

evo = Suzuki()
evolution = evo.convert(operator.to_pauli_op())  # bug... should work without `to_pauli_op` :)
print(evolution)
print(evolution.to_circuit_op())

ComposedOp([
  e^(-i*0.5 * ZZI),
  e^(-i*0.5 * XXI),
  e^(-i*0.5 * XXI),
  e^(-i*0.5 * ZZI)
])
     ┌──────────────┐┌──────────────┐┌──────────────┐┌──────────────┐
q_0: ┤0             ├┤0             ├┤0             ├┤0             ├
     │              ││              ││              ││              │
q_1: ┤1 Hamiltonian ├┤1 Hamiltonian ├┤1 Hamiltonian ├┤1 Hamiltonian ├
     │              ││              ││              ││              │
q_2: ┤2             ├┤2             ├┤2             ├┤2             ├
     └──────────────┘└──────────────┘└──────────────┘└──────────────┘


### Gradients

See later.

## 3. Simulation and evaluation

To simply evaluate an operator expression with plain matrix multiplication we have seen that we can use the `eval` method. To use Aer's simulator or a real backend we can use another object: the `CircuitSampler`.

The `CircuitSampler`, which contains a backend, takes as input an operator expression and returns an object where all circuits have been evaluated with the backend.

In [71]:
from qiskit.opflow import CircuitSampler
from qiskit.providers.aer import Aer

backend = Aer.get_backend('qasm_simulator')
sampler = CircuitSampler(backend)

circuit = QuantumCircuit(1)
circuit.ry(0.25, 0)
state = StateFn(circuit)

expectation = ~StateFn(H) @ state
sampled = sampler.convert(expectation)

print(sampled.eval())

(0.8603998842794964+0j)


In [72]:
print('before CircuitSampler:')
print(expectation)

sampled = sampler.convert(expectation)
print('after CircuitSampler:')
print(sampled)

before CircuitSampler:
ComposedOp([
  OperatorMeasurement(     ┌───┐
  q_0: ┤ H ├
       └───┘),
  CircuitStateFn(
       ┌──────────┐
  q_0: ┤ RY(0.25) ├
       └──────────┘
  )
])
after CircuitSampler:
ComposedOp([
  OperatorMeasurement(     ┌───┐
  q_0: ┤ H ├
       └───┘),
  DictStateFn({'0': 0.993140536379419, '1': 0.11692679333668567})
])


## All together 

Let's put all this knowledge together to compute the energy for the `EfficientSU2` ansatz for the H2 molecule.

In [83]:
import numpy as np
from qiskit.circuit.library import EfficientSU2
from qiskit.utils import QuantumInstance

# ansatz
circuit = EfficientSU2(2, reps=3)
params = circuit.ordered_parameters
state = StateFn(circuit)

# h2 operator in parity mapping and two qubit reductions
h2_op = -1.052373245772859 * (I ^ I) + 0.39793742484318045 * (I ^ Z) \
        - 0.39793742484318045 * (Z ^ I) - 0.01128010425623538 * (Z ^ Z) \
        + 0.18093119978423156 * (X ^ X)

# expecation value
expecation = ~StateFn(h2_op) @ state

# expectation computation like on real hardware
pauli = PauliExpectation()
pauli_expectation = pauli.convert(expectation)

# circuit sampler for executing
backend = QuantumInstance(Aer.get_backend('qasm_simulator'), shots=8192)
sampler = CircuitSampler(backend)

# some parameter values
values = [0] * circuit.num_parameters
bound_expecation = expectation.bind_parameters(dict(zip(params, values)))

# evaluate
sampled = sampler.convert(pauli_expectation)
energy = sampled.eval()
print('Energy:', energy)


Measured Observable is not composed of only Paulis, converting to Pauli representation, which can be expensive.


Energy: (0.857125285300588+0j)


## ListOp gymnastics

The `ListOp` is an extreme versatile tool. It allows to store any kind of operator objects, to perform the same set of operations on them, and to combine them with an arbitrary accumulation function upon evaluation.

1. Evaluation of multiple operators 

In [84]:
from qiskit.opflow import ListOp

np.set_printoptions(2)

vector_op = ListOp([I, X, Y, Z])
expectations = ~StateFn(vector_op) @ Zero
print('Vector:\n', expectations.eval())

matrix = []
for i in range(2):
    row = []
    for j in range(2):
        circuit = QuantumCircuit(1)
        circuit.ry(0.2 * 2 ** (i + j), 0)
        row.append(CircuitOp(circuit))
    matrix.append(ListOp(row))
matrix_op = ListOp(matrix)
print(matrix_op)
expectations = ~StateFn(matrix_op) @ One
print('Matrix:\n', np.array(expectations.eval()))

Vector:
 [(1+0j), 0.0, 0.0, (1+0j)]
ListOp([
  ListOp([
         ┌─────────┐
    q_0: ┤ RY(0.2) ├
         └─────────┘,
         ┌─────────┐
    q_0: ┤ RY(0.4) ├
         └─────────┘
  ]),
  ListOp([
         ┌─────────┐
    q_0: ┤ RY(0.4) ├
         └─────────┘,
         ┌─────────┐
    q_0: ┤ RY(0.8) ├
         └─────────┘
  ])
])
Matrix:
 [[1.  +0.j 0.98+0.j]
 [0.98+0.j 0.92+0.j]]


2. Using combo functions

In [32]:
# simple summed op
summed_op = ListOp([I, X, Y, Z], combo_fn=lambda x: sum(x))
expectations = ~StateFn(summed_op) @ Zero
print('SummedOp:\n', expectations.eval())

# multiplied op
mult_op = ListOp([I, X, Y, Z], combo_fn=lambda x: np.prod(x))
expectations = ~StateFn(mult_op) @ Zero
print('MultOp:\n', expectations.eval())

# max op
max_op = ListOp([I, X, Y, Z], combo_fn=lambda x: np.max(np.abs(x)))
expectations = ~StateFn(max_op) @ Zero
print('MaxOp:\n', expectations.eval())

# quadratic sum
def quadsum(x):
    return sum(x_i ** 2 for x_i in x)

quad_op = ListOp([I, X, Y, Z], combo_fn=quadsum)
expectations = ~StateFn(quad_op) @ Zero
print('QuadOp:\n', expectations.eval())

SummedOp:
 (2+0j)
MultOp:
 0j
MaxOp:
 1.0
QuadOp:
 (2+0j)
